1. Imports

In [120]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from google.colab import drive
import os
from google.colab import files

2. Localisation et ouverture des données

In [121]:
!ls /content/drive/MyDrive/deliveroo

Fri_01_Nov_2019_12_00_40_.html	Sat_1_May_2021_09_51_08_.html
Fri_11_Sep_2020_18_09_27_.html	Sat_23_Jan_2021_16_16_55_.html
Fri_12_Jun_2020_20_01_52_.html	Sat_26_Dec_2020_12_53_40_.html
Fri_12_Mar_2021_19_51_29_.html	Sat_26_Dec_2020_19_35_01_.html
Fri_13_Nov_2020_20_05_38_.html	Sat_29_May_2021_19_21_37_.html
Fri_15_Jan_2021_19_09_55_.html	Sat_2_Jan_2021_19_16_10_.html
Fri_16_Apr_2021_18_35_32_.html	Sat_3_Apr_2021_17_12_21_.html
Fri_18_Dec_2020_19_22_23_.html	Sat_3_Oct_2020_14_13_23_.html
Fri_18_Sep_2020_18_47_33_.html	Sat_8_May_2021_19_30_07_.html
Fri_19_Feb_2021_19_11_59_.html	Sat_9_Jan_2021_18_27_22_.html
Fri_19_Mar_2021_19_57_35_.html	Sun_11_Apr_2021_18_06_27_.html
Fri_21_Aug_2020_09_53_05_.html	Sun_11_Aug_2019_18_05_25_.html
Fri_23_Apr_2021_19_01_22_.html	Sun_14_Mar_2021_19_28_55_.html
Fri_26_Feb_2021_20_06_57_.html	Sun_20_Dec_2020_19_42_08_.html
Fri_26_Mar_2021_19_56_07_.html	Sun_22_Nov_2020_19_07_23_.html
Fri_28_Aug_2020_12_40_44_.html	Sun_24_Jan_2021_18_42_19_.html
Fri_28_May_202

In [122]:
DELIVEROO_PATH = "/content/drive/MyDrive/deliveroo/"

# Obtenir la liste des fichiers HTML
files = [f for f in os.listdir(DELIVEROO_PATH) if f.endswith('.html')]

In [123]:
# Charger le premier fichier HTML pour tester
file = files[0]

# Ouvrir et lire le fichier avec encodage UTF-8
with open(DELIVEROO_PATH + file, 'r', encoding='utf-8') as f:
    content = f.read()
    # Créer un objet BeautifulSoup pour parser le HTML
    soup = BeautifulSoup(content, 'html.parser')

print(f"Fichier chargé : {file}")

Fichier chargé : Fri_4_Jun_2021_18_12_55_.html


In [124]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<link href="https://cdn1.deliveroo.co.uk/assets/images/favicons/production/favicon-20e1155d7ebdf6585dbace1c09513152.ico" rel="shortcut icon"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>Deliveroo</title>
<style media="all" type="text/css">

    <link rel="stylesheet" media="screen" href="/stylesheets/stylesheets/emails.css" />

    </style>
</head>
<body style="-webkit-text-size-adjust:none">
<table bgcolor="#f7f7f7" border="0" cellpadding="0" cellspacing="0" id="backgroundtable" role="presentation" style="min-width:600px;" width="100%">
<tr>
<td align="center" valign="top">
<table border="0" cellpadding="0" cellspacing="0" class="section" role="presentation" width="100%">
<tr>
<td align="center" valign="top">
<span class="email

3. Fonctions de traitement des html

In [125]:
def extract_order_data(soup, filename):
    """Extrait les données de commande depuis le HTML"""

    order = {
        "order_datetime": None,
        "order_number": None,
        "delivery_fee": None,
        "order_total_paid": None
    }

    # 1. Extraction de la date depuis le nom du fichier
    # Le fichier s'appelle par exemple: Fri_4_Jun_2021_18_12_55_.html
    # On transforme en: Fri 4 Jun 2021 18:12:55
    parts = filename.replace('_', ' ').replace('.html', '').strip().split()
    if len(parts) >= 6:
        # Rejoindre la date et l'heure avec les deux-points
        date_str = ' '.join(parts[:-3]) + ' ' + ':'.join(parts[-3:])
        order["order_datetime"] = date_str

    # 2. Extraction du numéro de commande
    # On cherche dans tous les titres H2 le pattern "Commande n° XXXX"
    h2_tags = soup.find_all('h2')
    for h2 in h2_tags:
        text = h2.get_text()
        order_num_match = re.search(r'Commande n° (\d+)', text)
        if order_num_match:
            order["order_number"] = order_num_match.group(1)
            break

    # 3. Extraction des frais de livraison
    # On parcourt toutes les lignes de tableau pour trouver "Frais de livraison"
    all_tr = soup.find_all('tr')
    for tr in all_tr:
        tds = tr.find_all('td')
        if len(tds) >= 2:
            first_td_text = tds[0].get_text(strip=True)
            # Si la première colonne contient exactement "Frais de livraison"
            if first_td_text == 'Frais de livraison':
                # Extraire le prix de la deuxième colonne
                price_text = tds[1].get_text(strip=True)
                price_match = re.search(r'€([\d.]+)', price_text)
                if price_match:
                    order["delivery_fee"] = float(price_match.group(1))
                break

    # 4. Extraction du total payé
    # On cherche les paragraphes avec la classe "total"
    for tr in all_tr:
        total_p = tr.find_all('p', class_='total')
        if len(total_p) >= 2:
            # Le prix est dans le deuxième paragraphe
            price_text = total_p[1].get_text(strip=True)
            price_match = re.search(r'€([\d.]+)', price_text)
            if price_match:
                order["order_total_paid"] = float(price_match.group(1))
            break

    return order

In [126]:
# Tester
filename = "Fri_4_Jun_2021_18_12_55_.html"  # Remplace par le vrai nom de ton fichier
order_data = extract_order_data(soup, filename)
print(json.dumps(order_data, indent=2, ensure_ascii=False))

{
  "order_datetime": "Fri 4 Jun 2021 18:12:55",
  "order_number": "1833",
  "delivery_fee": 3.5,
  "order_total_paid": 16.25
}


In [127]:
def extract_restaurant_data(soup):
    """Extrait les données du restaurant depuis le HTML"""

    restaurant = {
        "name": None,
        "address": None,
        "city": None,
        "postcode": None,
        "phone_number": None
    }

    # 1. Trouver les tables qui contiennent les infos du restaurant
    # Le restaurant est dans une table avec class="fluid", width="200" et align="left"
    fluid_tables = soup.find_all('table', class_='fluid', width='200', align='left')

    # 2. Parcourir chaque table trouvée
    for table in fluid_tables:
        ps = table.find_all('p')

        # 3. Vérifier qu'on a au moins 5 paragraphes (nom, adresse, ville, code postal, téléphone)
        if len(ps) >= 5:
            first_p_style = ps[0].get('style', '')
            # Le restaurant est identifié par le premier paragraphe en gras (bolder)
            if 'bolder' in first_p_style:
                # Extraire les 5 infos du restaurant dans l'ordre
                restaurant["name"] = ps[0].get_text(strip=True)
                restaurant["address"] = ps[1].get_text(strip=True)
                restaurant["city"] = ps[2].get_text(strip=True)
                restaurant["postcode"] = ps[3].get_text(strip=True)
                restaurant["phone_number"] = ps[4].get_text(strip=True)
                break

    return restaurant

In [128]:
# Tester
restaurant_data = extract_restaurant_data(soup)
print(json.dumps(restaurant_data, indent=2, ensure_ascii=False))

{
  "name": "🔥BURGER KING®",
  "address": "622 ROUTE DU BORD DE MER",
  "city": "Nice",
  "postcode": "06270",
  "phone_number": "+33422020043"
}


In [129]:
def extract_customer_data(soup):
    """Extrait les données du client depuis le HTML"""

    customer = {
        "name": None,
        "address": None,
        "city": None,
        "postcode": None,
        "phone_number": None
    }

    # 1. Trouver les tables qui peuvent contenir les infos du client
    # Le client est dans une table avec class="fluid" et width="200"
    fluid_tables = soup.find_all('table', class_='fluid', width='200')

    # 2. Parcourir chaque table trouvée
    for table in fluid_tables:
        # Chercher les paragraphes avec class="alignleft"
        ps = table.find_all('p', class_='alignleft')

        # 3. Vérifier qu'on a au moins 5 paragraphes (nom, adresse, ville, code postal, téléphone)
        if len(ps) >= 5:
            # Le client est identifié par text-align:right dans le style
            first_p_style = ps[0].get('style', '')
            if 'text-align:right' in first_p_style:
                # Extraire les 5 infos du client dans l'ordre
                customer["name"] = ps[0].get_text(strip=True)
                customer["address"] = ps[1].get_text(strip=True)
                customer["city"] = ps[2].get_text(strip=True)
                customer["postcode"] = ps[3].get_text(strip=True)
                customer["phone_number"] = ps[4].get_text(strip=True)
                break

    return customer

In [130]:
# Tester
customer_data = extract_customer_data(soup)
print(json.dumps(customer_data, indent=2, ensure_ascii=False))

{
  "name": "Regis amichia",
  "address": "4 Chemin de Saint-Laurent du Var",
  "city": "Nice",
  "postcode": "06800",
  "phone_number": "+33 6 15 78 51 52"
}


In [131]:
def extract_order_items(soup):
    """Extrait tous les articles commandés depuis le HTML"""

    order_items = []

    # 1. Trouver toutes les tables contenant des articles
    # Les articles sont dans des tables avec role="listitem"
    listitem_tables = soup.find_all('table', {'role': 'listitem'})

    # 2. Parcourir chaque table d'articles
    for table in listitem_tables:
        # Trouver toutes les lignes (il peut y avoir plusieurs articles par table)
        trs = table.find_all('tr')

        # 3. Parcourir chaque ligne de la table
        for tr in trs:
            tds = tr.find_all('td')

            # Vérifier qu'on a 3 colonnes: quantité, nom, prix
            if len(tds) >= 3:
                # Extraire la quantité (format "1x", "2x", etc.)
                quantity_text = tds[0].get_text(strip=True)
                quantity_match = re.match(r'(\d+)x', quantity_text)

                # Extraire le nom de l'article (premier paragraphe uniquement)
                # On évite les sous-éléments en ne prenant que le premier <p> direct
                name = None
                for child in tds[1].children:
                    if child.name == 'p':
                        name = child.get_text(strip=True)
                        break

                # Extraire le prix (format "12,55 €")
                price_text = tds[2].get_text(strip=True)
                price_match = re.search(r'([\d,]+)\s*€', price_text)

                # Si on a réussi à extraire les 3 infos, ajouter l'article
                if quantity_match and name and price_match:
                    quantity = int(quantity_match.group(1))
                    # Remplacer la virgule par un point pour le format float
                    price = float(price_match.group(1).replace(',', '.'))

                    order_items.append({
                        "name": name,
                        "quantity": quantity,
                        "price": price
                    })

    return order_items

In [132]:
order_items = extract_order_items(soup)
print(json.dumps(order_items, indent=2, ensure_ascii=False))

[
  {
    "name": "Menu Double Cheese Bacon XXL",
    "quantity": 1,
    "price": 12.55
  }
]


4. Fonctions d'extraction (réunies dans une case)

In [133]:
def extract_order_data(soup, filename):
    """Extrait les données de commande depuis le HTML"""

    order = {
        "order_datetime": None,
        "order_number": None,
        "delivery_fee": None,
        "order_total_paid": None
    }

    # 1. Extraction de la date depuis le nom du fichier
    # Le fichier s'appelle par exemple: Fri_4_Jun_2021_18_12_55_.html
    # On transforme en: Fri 4 Jun 2021 18:12:55
    parts = filename.replace('_', ' ').replace('.html', '').strip().split()
    if len(parts) >= 6:
        date_str = ' '.join(parts[:-3]) + ' ' + ':'.join(parts[-3:])
        order["order_datetime"] = date_str

    # 2. Extraction du numéro de commande
    # On cherche dans tous les titres H2 le pattern "Commande n° XXXX"
    h2_tags = soup.find_all('h2')
    for h2 in h2_tags:
        text = h2.get_text()
        order_num_match = re.search(r'Commande n° (\d+)', text)
        if order_num_match:
            order["order_number"] = order_num_match.group(1)
            break

    # 3. Extraction des frais de livraison
    # On parcourt toutes les lignes de tableau pour trouver "Frais de livraison"
    all_tr = soup.find_all('tr')
    for tr in all_tr:
        tds = tr.find_all('td')
        if len(tds) >= 2:
            first_td_text = tds[0].get_text(strip=True)
            if first_td_text == 'Frais de livraison':
                price_text = tds[1].get_text(strip=True)
                price_match = re.search(r'€([\d.]+)', price_text)
                if price_match:
                    order["delivery_fee"] = float(price_match.group(1))
                break

    # 4. Extraction du total payé
    # On cherche les paragraphes avec la classe "total"
    for tr in all_tr:
        total_p = tr.find_all('p', class_='total')
        if len(total_p) >= 2:
            price_text = total_p[1].get_text(strip=True)
            price_match = re.search(r'€([\d.]+)', price_text)
            if price_match:
                order["order_total_paid"] = float(price_match.group(1))
            break

    return order


def extract_restaurant_data(soup):
    """Extrait les données du restaurant depuis le HTML"""

    restaurant = {
        "name": None,
        "address": None,
        "city": None,
        "postcode": None,
        "phone_number": None
    }

    # 1. Trouver les tables qui contiennent les infos du restaurant
    # Le restaurant est dans une table avec class="fluid", width="200" et align="left"
    fluid_tables = soup.find_all('table', class_='fluid', width='200', align='left')

    # 2. Parcourir chaque table trouvée
    for table in fluid_tables:
        ps = table.find_all('p')

        # 3. Vérifier qu'on a au moins 5 paragraphes
        if len(ps) >= 5:
            first_p_style = ps[0].get('style', '')
            # Le restaurant est identifié par le premier paragraphe en gras
            if 'bolder' in first_p_style:
                # Extraire les 5 infos du restaurant (on garde les emojis)
                restaurant["name"] = ps[0].get_text(strip=True)
                restaurant["address"] = ps[1].get_text(strip=True)
                restaurant["city"] = ps[2].get_text(strip=True)
                restaurant["postcode"] = ps[3].get_text(strip=True)
                restaurant["phone_number"] = ps[4].get_text(strip=True)
                break

    return restaurant


def extract_customer_data(soup):
    """Extrait les données du client depuis le HTML"""

    customer = {
        "name": None,
        "address": None,
        "city": None,
        "postcode": None,
        "phone_number": None
    }

    # 1. Trouver les tables qui peuvent contenir les infos du client
    # Le client est dans une table avec class="fluid" et width="200"
    fluid_tables = soup.find_all('table', class_='fluid', width='200')

    # 2. Parcourir chaque table trouvée
    for table in fluid_tables:
        ps = table.find_all('p', class_='alignleft')

        # 3. Vérifier qu'on a au moins 5 paragraphes
        if len(ps) >= 5:
            first_p_style = ps[0].get('style', '')
            # Le client est identifié par text-align:right dans le style
            if 'text-align:right' in first_p_style:
                # Extraire les 5 infos du client
                customer["name"] = ps[0].get_text(strip=True)
                customer["address"] = ps[1].get_text(strip=True)
                customer["city"] = ps[2].get_text(strip=True)
                customer["postcode"] = ps[3].get_text(strip=True)
                customer["phone_number"] = ps[4].get_text(strip=True)
                break

    return customer


def extract_order_items(soup):
    """Extrait tous les articles commandés depuis le HTML"""

    order_items = []

    # 1. Trouver toutes les tables contenant des articles
    # Les articles sont dans des tables avec role="listitem"
    listitem_tables = soup.find_all('table', {'role': 'listitem'})

    # 2. Parcourir chaque table d'articles
    for table in listitem_tables:
        # Trouver toutes les lignes (il peut y avoir plusieurs articles)
        trs = table.find_all('tr')

        # 3. Parcourir chaque ligne de la table
        for tr in trs:
            tds = tr.find_all('td')

            # Vérifier qu'on a 3 colonnes: quantité, nom, prix
            if len(tds) >= 3:
                # Extraire la quantité (format "1x", "2x", etc.)
                quantity_text = tds[0].get_text(strip=True)
                quantity_match = re.match(r'(\d+)x', quantity_text)

                # Extraire le nom (premier paragraphe uniquement)
                name = None
                for child in tds[1].children:
                    if child.name == 'p':
                        name = child.get_text(strip=True)
                        break

                # Extraire le prix (format "12,55 €")
                price_text = tds[2].get_text(strip=True)
                price_match = re.search(r'([\d,]+)\s*€', price_text)

                # Si on a réussi à extraire les 3 infos, ajouter l'article
                if quantity_match and name and price_match:
                    quantity = int(quantity_match.group(1))
                    price = float(price_match.group(1).replace(',', '.'))

                    order_items.append({
                        "name": name,
                        "quantity": quantity,
                        "price": price
                    })

    return order_items

5. Traitement de tous les fichiers

In [135]:
# Lister tous les fichiers HTML du dossier
files = [f for f in os.listdir(DELIVEROO_PATH) if f.endswith('.html')]
all_orders = []

print(f"Traitement de {len(files)} fichiers HTML...\n")

# Parcourir chaque fichier HTML
for i, file in enumerate(files, 1):
    try:
        # Ouvrir et lire le fichier avec encodage UTF-8
        with open(DELIVEROO_PATH + file, 'r', encoding='utf-8') as f:
            content = f.read()
            soup = BeautifulSoup(content, 'html.parser')

            # Appeler les 4 fonctions d'extraction
            order_data = extract_order_data(soup, file)
            restaurant_data = extract_restaurant_data(soup)
            customer_data = extract_customer_data(soup)
            order_items = extract_order_items(soup)

            # Combiner toutes les données en un seul objet JSON
            complete_order = {
                "order": order_data,
                "restaurant": restaurant_data,
                "customer": customer_data,
                "order_items": order_items
            }

            all_orders.append(complete_order)

            # Afficher la progression tous les 10 fichiers
            if i % 10 == 0:
                print(f"✓ {i}/{len(files)} fichiers traités")

    except Exception as e:
        print(f"✗ Erreur sur {file}: {e}")

print(json.dumps(all_orders, indent=2, ensure_ascii=False))

Traitement de 82 fichiers HTML...

✓ 10/82 fichiers traités
✓ 20/82 fichiers traités
✓ 30/82 fichiers traités
✓ 40/82 fichiers traités
✓ 50/82 fichiers traités
✓ 60/82 fichiers traités
✓ 70/82 fichiers traités
✓ 80/82 fichiers traités
[
  {
    "order": {
      "order_datetime": "Fri 4 Jun 2021 18:12:55",
      "order_number": "1833",
      "delivery_fee": 3.5,
      "order_total_paid": 16.25
    },
    "restaurant": {
      "name": "🔥BURGER KING®",
      "address": "622 ROUTE DU BORD DE MER",
      "city": "Nice",
      "postcode": "06270",
      "phone_number": "+33422020043"
    },
    "customer": {
      "name": "Regis amichia",
      "address": "4 Chemin de Saint-Laurent du Var",
      "city": "Nice",
      "postcode": "06800",
      "phone_number": "+33 6 15 78 51 52"
    },
    "order_items": [
      {
        "name": "Menu Double Cheese Bacon XXL",
        "quantity": 1,
        "price": 12.55
      }
    ]
  },
  {
    "order": {
      "order_datetime": "Tue 4 May 2021 11:25:5

6. Sauvegarde du json

In [137]:
files.download('/content/deliveroo_orders.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>